In [1]:
# Basic Python Libs - Not everything will be used here...
import os
import sys
import importlib
import csv
import pydotplus
import plotly
import numpy as np
import pandas as pd
import pandas_profiling
import seaborn as sns
import matplotlib.cm as cm
from scipy import stats
from matplotlib import pyplot as plt
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV, train_test_split, learning_curve
from sklearn.metrics import classification_report,confusion_matrix, roc_curve, auc, roc_auc_score, precision_recall_curve
import joblib
np.set_printoptions(threshold=sys.maxsize)

# Own Imports
from Parameters import *
from Classifier_Trainer import *
from Classifier_Validation_Plots import *

c:\users\oli\appdata\local\programs\python\python37-32\lib\site-packages\sklearn\externals\six.py:31: DeprecationWarning:

The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).



Now I'm going to train a model. Before I can do that I need to label and encode the categorial variables (Since categorial features are not supported in sklearn...)

In [2]:
# Paths, functions and other useful stuff
data_path = "..\\Data\\"
file_name = "challenge_data.csv"

classifier_path = classifier_name[0]
if not os.path.exists(classifier_path):
    os.makedirs(classifier_path)
model_path = classifier_path+"\Model\\"
if not os.path.exists(model_path):
    os.makedirs(model_path)
plot_path = classifier_path+"\Plots"
if not os.path.exists(plot_path):
    os.makedirs(plot_path)
tree_path = plot_path+"\Decision_Trees\\"
if not os.path.exists(tree_path):
    os.makedirs(tree_path)
correlation_path = plot_path+"\Correlations"
if not os.path.exists(correlation_path):
    os.makedirs(correlation_path)
training_path = plot_path+"\Training\\"
if not os.path.exists(training_path):
    os.makedirs(training_path)
validation_path = plot_path+"\Validation\\"
if not os.path.exists(validation_path):
    os.makedirs(validation_path)
visualization_path = plot_path+"\Visualization\\"
if not os.path.exists(visualization_path):
    os.makedirs(visualization_path)

In [3]:
# Data import and dummy variables
df = pd.read_csv(data_path+file_name, delimiter=',')
df = df.drop(columns=['Unnamed: 0'])

# Regrouping
df['brand']=np.where((df['brand'] == 'Brand78') |
                     (df['brand'] == 'Brand75') |
                     (df['brand'] == 'Brand56') |
                     (df['brand'] == 'Brand71') |
                     (df['brand'] == 'Brand70') |
                     (df['brand'] == 'Brand65') |
                     (df['brand'] == 'Brand82') |
                     (df['brand'] == 'Brand69') |
                     (df['brand'] == 'Brand51') |
                     (df['brand'] == 'Brand64') |
                     (df['brand'] == 'Brand37') |
                     (df['brand'] == 'Brand44') |
                     (df['brand'] == 'Brand19') |
                     (df['brand'] == 'Brand35') |
                     (df['brand'] == 'Brand33') |
                     (df['brand'] == 'Brand54') |
                     (df['brand'] == 'Brand25') |
                     (df['brand'] == 'Brand13') |
                     (df['brand'] == 'Brand45') |
                     (df['brand'] == 'Brand60') |
                     (df['brand'] == 'Brand59') |
                     (df['brand'] == 'Brand72') |
                     (df['brand'] == 'Brand26') |
                     (df['brand'] == 'Brand50') |
                     (df['brand'] == 'Brand63') |
                     (df['brand'] == 'Brand41') |
                     (df['brand'] == 'Brand68') |
                     (df['brand'] == 'Brand46') |
                     (df['brand'] == 'Brand9') |
                     (df['brand'] == 'Brand5') |
                     (df['brand'] == 'Brand67') |
                     (df['brand'] == 'Brand38') |
                     (df['brand'] == 'Brand49') |
                     (df['brand'] == 'Brand30') |
                     (df['brand'] == 'Brand12') |
                     (df['brand'] == 'Brand31') |
                     (df['brand'] == 'Brand40') |
                     (df['brand'] == 'Brand52') |
                     (df['brand'] == 'Brand47') |
                     (df['brand'] == 'Brand39') |
                     (df['brand'] == 'Brand10') |
                     (df['brand'] == 'Brand79') |
                     (df['brand'] == 'Brand83'),
                     'bad',
                     df['brand'])

df['brand']=np.where((df['brand'] == 'Brand17') |
                     (df['brand'] == 'Brand21') |
                     (df['brand'] == 'Brand4') |
                     (df['brand'] == 'Brand14') |
                     (df['brand'] == 'Brand16') |
                     (df['brand'] == 'Brand77') |
                     (df['brand'] == 'Brand8') |
                     (df['brand'] == 'Brand0') |
                     (df['brand'] == 'Brand80') |
                     (df['brand'] == 'Brand58') |
                     (df['brand'] == 'Brand53') |
                     (df['brand'] == 'Brand43') |
                     (df['brand'] == 'Brand22') |
                     (df['brand'] == 'Brand74') |
                     (df['brand'] == 'Brand57') |
                     (df['brand'] == 'Brand55') |
                     (df['brand'] == 'Brand18') |
                     (df['brand'] == 'Brand20') |
                     (df['brand'] == 'Brand28'),
                     'medium',
                     df['brand'])

df['brand']=np.where((df['brand'] == 'Brand1') |
                     (df['brand'] == 'Brand11') |
                     (df['brand'] == 'Brand76') |
                     (df['brand'] == 'Brand29') |
                     (df['brand'] == 'Brand42') |
                     (df['brand'] == 'Brand7') |
                     (df['brand'] == 'Brand48') |
                     (df['brand'] == 'Brand6') |
                     (df['brand'] == 'Brand27') |
                     (df['brand'] == 'Brand32') |
                     (df['brand'] == 'Brand15') |
                     (df['brand'] == 'Brand3') |
                     (df['brand'] == 'Brand23') |
                     (df['brand'] == 'Brand81') |
                     (df['brand'] == 'Brand62') |
                     (df['brand'] == 'Brand61') |
                     (df['brand'] == 'Brand66') |
                     (df['brand'] == 'Brand24') |
                     (df['brand'] == 'Brand34'),
                     'good',
                     df['brand'])

# print(df['brand'].unique())
# for item in df['brand'].unique():
#     print(len(df['brand'][df.brand == item])/len(df.brand))


df['color']=np.where((df['color'] == 'Metal') |
                     (df['color'] == 'Black') |
                     (df['color'] == 'White') |
                     (df['color'] == 'Pink'),
                     'bad',
                     df['color'])

df['color']=np.where((df['color'] == 'Grey') |
                     (df['color'] == 'Turquoise') |
                     (df['color'] == 'Beige') |
                     (df['color'] == 'Multicolor') |
                     (df['color'] == 'Brown') |
                     (df['color'] == 'Yellow') |
                     (df['color'] == 'Violet'),
                     'medium',
                     df['color'])

df['color']=np.where((df['color'] == 'Green') |
                     (df['color'] == 'Red') |
                     (df['color'] == 'Orange'),
                     'good',
                     df['color'])

# print(df['color'].unique())
# for item in df['color'].unique():
#     print(len(df['color'][df.color == item])/len(df.color))


df['product_group']=np.where((df['product_group'] == 'Denim jacket') |
                     (df['product_group'] == 'Caban') |
                     (df['product_group'] == 'Trench coat') |
                     (df['product_group'] == 'Short Coat') |
                     (df['product_group'] == 'Blouson') |
                     (df['product_group'] == 'Winter coat') |
                     (df['product_group'] == 'Leather Jacket') |
                     (df['product_group'] == 'Field jacket') |
                     (df['product_group'] == 'Functional jacket') |
                     (df['product_group'] == 'Blazer casual') |
                     (df['product_group'] == 'Chino slim fit') |
                     (df['product_group'] == 'Winter jacket') |
                     (df['product_group'] == 'Light coat') |
                     (df['product_group'] == 'Light jacket') |
                     (df['product_group'] == 'Parka') |
                     (df['product_group'] == 'PU-Jacket') |
                     (df['product_group'] == 'Down jacket') |
                     (df['product_group'] == 'Outdoor Vest') |
                     (df['product_group'] == 'Belt casual') |
                     (df['product_group'] == 'Cardigan') |
                     (df['product_group'] == 'T-shirt long sleeves') |
                     (df['product_group'] == 'Blazer Knit') |
                     (df['product_group'] == 'Sweatpants') |
                     (df['product_group'] == 'Cloth pants') |
                     (df['product_group'] == 'Indoor Vest'),
                     'bad',
                     df['product_group'])

df['product_group']=np.where((df['product_group'] == 'Medium fit') |
                     (df['product_group'] == 'Briefs') |
                     (df['product_group'] == 'Polo shirt longsleeves') |
                     (df['product_group'] == 'Slim fit') |
                     (df['product_group'] == 'Regular fit') |
                     (df['product_group'] == 'T-shirt Print') |
                     (df['product_group'] == 'Sweatshirt') |
                     (df['product_group'] == 'Boxers') |
                     (df['product_group'] == 'Troyer') |
                     (df['product_group'] == 'Polo shirt shortsleeves') |
                     (df['product_group'] == 'Hoodie') |
                     (df['product_group'] == 'V-Neck') |
                     (df['product_group'] == 'T-shirt Basic') |
                     (df['product_group'] == 'C-Neck'),
                     'medium',
                     df['product_group'])

df['product_group']=np.where((df['product_group'] == 'T-shirt striped / patterned') |
                     (df['product_group'] == 'Chino shorts') |
                     (df['product_group'] == 'Undershirt') |
                     (df['product_group'] == 'Chino regular fit') |
                     (df['product_group'] == 'Turtle neck') |
                     (df['product_group'] == 'Swim shorts') |
                     (df['product_group'] == 'Sweat jacket') |
                     (df['product_group'] == 'Trunks') |
                     (df['product_group'] == 'Casual shirt short sleeves') |
                     (df['product_group'] == 'Jeans shorts') |
                     (df['product_group'] == 'Business shirt short sleeves') |
                     (df['product_group'] == 'Cargo Shorts'),
                     'good',
                     df['product_group'])

# print(df['product_group'].unique())
# for item in df['product_group'].unique():
#     print(len(df['product_group'][df.product_group == item])/len(df.product_group))

    
df['category']=np.where((df['category'] == 'Coats') |
                     (df['category'] == 'Leather Jackets') |
                     (df['category'] == 'Blazers') |
                     (df['category'] == 'Jackets') |
                     (df['category'] == 'Vest') |
                     (df['category'] == 'Belts'),
                     'bad',
                     df['category'])

df['category']=np.where((df['category'] == 'Pants') |
                     (df['category'] == 'T-Shirts') |
                     (df['category'] == 'Knitted tops') |
                     (df['category'] == 'Shirts business'),
                     'medium',
                     df['category'])

df['category']=np.where((df['category'] == 'Sweaters') |
                     (df['category'] == 'Underwear') |
                     (df['category'] == 'Shorts') |
                     (df['category'] == 'Shirt short sleeves'),
                     'good',
                     df['category'])

# print(df['category'].unique())
# for item in df['category'].unique():
#     print(len(df['category'][df.category == item])/len(df.category))    

cat_cols = ['category', 'product_group', 'color', 'brand', 'shipping_country']
df_dummies = pd.get_dummies(df, prefix_sep="_",
                              columns=cat_cols)

size_values = ['S', 'M', 'L', 'XL', 'XXL']
size_label = ['0', '1', '2', '3', '4']
df_dummies = df_dummies.replace(to_replace=size_values, value=size_label)
#print(df_dummies.columns)

Now we have our dummy variables and we labled our size feature accordingly.
In order to train our model, we will need to split our dataset into at least one training and one testing sample. I would invest more time in a better splitting, but given the timeframe I'll settle with a very simple splitting.
Next up we define our features.

In [4]:
#Feature_List = df_dummies.columns.tolist()
#Feature_List.remove('itret')
#Feature_List.remove('date_shipped')
#print(Feature_List)

X = df_dummies[Feature_List] # Indepent variables
y = df_dummies['itret'] # Dependent Variables

# Split
X_dev, X_eval, y_dev, y_eval = train_test_split(X, y, test_size=reserve_fraction, random_state=42)
print("\nReserving " + str(format(reserve_fraction*100, '.2f')) + "% Of Events For Later Use: " + str(format(len(X_eval), ',.0f')))
print("\nUsing " + str(format((1-reserve_fraction)*100, '.2f')) + "% Of Events For Splitting: " + str(format(len(X_dev), ',.0f')))
X_train, X_test, y_train, y_test = train_test_split(X_dev, y_dev, test_size=test_fraction, random_state=random_seeds[0])
print("\nUsing " + str(format((1-reserve_fraction)*(1-test_fraction)*100, '.2f')) + "% Of Events For Training: " + str(format(len(X_train), ',.0f')))
print("\nUsing " + str(format((1-reserve_fraction)*test_fraction*100, '.2f')) + "% Of Events For Testing: " + str(format(len(X_test), ',.0f')))
print("\nData Has Been Split...")


Reserving 5.00% Of Events For Later Use: 7,344

Using 95.00% Of Events For Splitting: 139,522

Using 47.50% Of Events For Training: 69,761

Using 47.50% Of Events For Testing: 69,761

Data Has Been Split...


In [5]:
# parameters = {
#     "loss":["deviance"],
#     "min_samples_leaf": np.linspace(0.05, 0.2, 4),
#     "max_depth": range(2,6),
#     "criterion": ["friedman_mse"],
#     "subsample":[0.8, 0.9, 1.0],
#     "learning_rate": [0.01, 0.05, 0.1],
#     "n_estimators": np.arange(50, 300, 50)
#     }


# # run randomized search
# print("Defining the Gridsearch")
# clf = GridSearchCV(GradientBoostingClassifier(), parameters, scoring='roc_auc', cv=3, n_jobs=4, verbose=2)
# print("Training on all gridpoints...")
# clf.fit(X_train, y_train)
# print('Gradient boosting trees best params:', clf.best_params_)
# print('Gradient boosting trees score:', clf.best_score_)

Define our model, train it and evaluate.

In [6]:
GB_clfs = {}
GB_clfs[str(classifier_name[0])] = GradientBoostingClassifier(max_depth=max_depth, 
                                                              min_samples_leaf=min_samples_leaf, 
                                                              random_state=0, subsample=subsample, 
                                                              n_estimators=n_estimators, 
                                                              learning_rate=learning_rate, 
                                                              verbose=True)
clfs=[]
print("\nCrosstraining - 0")
classifier_training(X_train, y_train, X_test, y_test, clfs, 0, 
                    GB_clfs[classifier_name[0]], model_path, tree_path, training_path, False)
print("---"*42)
print("Crosstraining - 1")
classifier_training(X_train, y_train, X_test, y_test,clfs,  1, 
                    GB_clfs[classifier_name[0]], model_path, tree_path, training_path, False)
print("\nFinished Training...")
print("---"*42)
clfs[0].set_params(verbose=False)
clfs[1].set_params(verbose=False)


Crosstraining - 0
      Iter       Train Loss   Remaining Time 
         1           1.3665           13.29s
         2           1.3501           12.67s
         3           1.3362           12.93s
         4           1.3236           13.46s
         5           1.3132           13.15s
         6           1.3035           12.95s
         7           1.2957           12.68s
         8           1.2881           12.64s
         9           1.2818           12.46s
        10           1.2763           12.16s
        20           1.2408           11.53s
        30           1.2261           11.00s
        40           1.2182           10.39s
        50           1.2131            9.84s
        60           1.2101            9.34s
        70           1.2079            8.88s
        80           1.2062            8.37s
        90           1.2051            7.80s
       100           1.2039            7.31s
       200           1.1961            2.39s
Safing trained model...
Decision Tr

GradientBoostingClassifier(criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=5,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=0.05, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=250,
                           n_iter_no_change=None, presort='auto',
                           random_state=0, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=False,
                           warm_start=False)

In [7]:
Ntree_ROC_Curve_Figname = validation_path + str(classifier_name[0]) + "_Tree_" + str(len(clfs[0].estimators_)) + "_ROC.png"
plot_Ntree_ROC_curve(clfs[0], (X_train,y_train),(X_test,y_test), Ntree_ROC_Curve_Figname)
Ntree_PR_Curve_Figname = validation_path + str(classifier_name[0]) + "_Tree_" + str(len(clfs[0].estimators_)) + "_PR.png"
plot_Ntree_PR_curve(clfs[0], (X_train,y_train),(X_test,y_test), Ntree_PR_Curve_Figname)

# Reference For Some Of These Numbers
# P - condition positive, the number of real positive cases in the data
# N - condition negative, the number of real negative cases in the data

# TP - true positive, eqv. with hit

# TN - true negative,  eqv. with correct rejection
# FP - false positive, eqv. with false alarm, Type I error
# FN - false negative,  eqv. with miss, Type II error

# PPV - Precision = TP / (TP + FP) = 1 - FDR, positive predictive value
# TPR - Recall = TP / (TP + FN) = 1 - FNR, true positive rate
# FNR - Miss Rate = FN / (FN + TP) = 1 - TPR, false negative rate
# FDR - False Discovery Rate = FP / (FP + TP)
# ACC - Accuracy = (TP + TN) / (TP + TN + FP + FN)
# F betta - F betta Score = (1+ betta**2) * ( PPV * TPR) / (betta**2 * PPV + TPR), is the harmonic mean of precision and recall

# If I ever want to plot multiple clfs
#fig, axes = plt.subplots(nrows=len(clfs), sharex=True)
#for clf, ax in zip(clfs, axes):  
#	plot_learning_curve(clf, "Learning curves", X_dev, y_dev, scoring='roc_auc', n_jobs=7, cv=4, ax=ax, xlabel=False)
#axes[0].legend(loc="best")
#axes[-1].set_xlabel("Training examples")

print("Plotting Performance vs. Size Of Training Set...")
fig, axis = plt.subplots(nrows=1, sharex=True)
plot_learning_curve(clfs[0], "Learning curves", X_dev, y_dev, n_jobs=n_jobs, cv=cv, ax=axis)
axis.legend(loc="best")
axis.set_xlabel("Training examples")
fig_name = validation_path + str(classifier_name[0]) + "_Score_Validation" + ".png"
plt.savefig(fig_name)
plt.close()
print("\nFinished Validation...")
print("---"*42)

Highest ROC-AUC At Tree: 250
Highest PR-AUC At Tree: 250
Plotting Performance vs. Size Of Training Set...
[learning_curve] Training set sizes: [ 9301 18602 27904 37205 46507 55808 65109 74411 83712 93014]


[Parallel(n_jobs=4)]: Using backend LokyBackend with 4 concurrent workers.



Finished Validation...
------------------------------------------------------------------------------------------------------------------------------


[Parallel(n_jobs=4)]: Done  30 out of  30 | elapsed:  1.8min finished
